<a href="https://colab.research.google.com/github/hyunicecream/Natural-Language-Processing-NLP-/blob/main/SKT_KOBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Aug  8 20:15:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    38W / 300W |  10743MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-08_414jc
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-08_414jc


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
0##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [ ]:
import pandas as pd
#dataset_train = pd.read_csv("/content/drive/MyDrive/공모전/open/morenews_cleaned_train.csv")
#dataset_test = pd.read_csv("/content/drive/MyDrive/공모전/open/morenews_cleaned_test.csv")
#dataset_test = nlp.data.TSVDataset("/content/drive/MyDrive/공모전/open/morenews_cleaned_test.csv, field_indices=[1,2], num_discard_samples=1)
dataset_train = pd.read_csv('/content/drive/MyDrive/공모전/open/train_otk_pororo_ver01.csv')
dataset_test = pd.read_csv('/content/drive/MyDrive/공모전/open/test_data.csv')

In [ ]:
from sklearn.model_selection import train_test_split
#train, test = train_test_split(dataset_train, test_size= 0.3, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(dataset_train['title'], dataset_train['topic_idx'], test_size= 0.3, random_state=42, stratify=dataset_train['topic_idx'])
print("train.shape is :", len(train))
print("test.shape is :", len(test))

train.shape is : 31957
test.shape is : 13697


In [ ]:
dataset_train.isnull().sum()

Unnamed: 0    0
title         0
topic_idx     0
dtype: int64

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
'''
class BERTDataset(Dataset):
    def __init__(self, dataset, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([title]) for title in dataset['title'].tolist()]
        self.labels = [np.int32(label) for label in dataset['topic_idx'].tolist()]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))
'''

"\nclass BERTDataset(Dataset):\n    def __init__(self, dataset, bert_tokenizer, max_len,\n                 pad, pair):\n        transform = nlp.data.BERTSentenceTransform(\n            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)\n\n        self.sentences = [transform([title]) for title in dataset['title'].tolist()]\n        self.labels = [np.int32(label) for label in dataset['topic_idx'].tolist()]\n\n    def __getitem__(self, i):\n        return (self.sentences[i] + (self.labels[i], ))\n\n    def __len__(self):\n        return (len(self.labels))\n"

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, x, y, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([title]) for title in x.tolist()]
        self.labels = [np.int32(label) for label in y.tolist()]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
## Setting parameters
max_len = 30
batch_size = 64
warmup_ratio = 0.1
num_epochs = 6
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#data_train = BERTDataset(dataset_train, tok, max_len, True, False)
#data_test = BERTDataset(dataset_train, tok, max_len, True, False)
data_train = BERTDataset(x_train, y_train, tok, max_len, True, False)
data_test = BERTDataset(x_test, y_test, tok, max_len, True, False)
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5, shuffle=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.0409023761749268 train acc 0.09375
epoch 1 batch id 201 loss 0.2128024399280548 train acc 0.7520988805970149
epoch 1 batch id 401 loss 0.23617172241210938 train acc 0.8491661471321695

epoch 1 train acc 0.8684843117408907


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.9512154266723842


...


epoch 4 test acc 0.9518050493138938


epoch 5 batch id 1 loss 0.02710290253162384 train acc 0.984375
epoch 5 batch id 201 loss 0.01327532809227705 train acc 0.9919931592039801
epoch 5 batch id 401 loss 0.004940832499414682 train acc 0.9915445760598504

epoch 5 train acc 0.9912702429149798



epoch 5 test acc 0.9540898370497428


epoch 6 batch id 1 loss 0.12818486988544464 train acc 0.984375
epoch 6 batch id 201 loss 0.003683418268337846 train acc 0.994247512437811
epoch 6 batch id 401 loss 0.005115378648042679 train acc 0.9939993765586035

epoch 6 train acc 0.9941801619433198



epoch 6 test acc 0.9538620283018868


In [ ]:
#torch.save(model_kobert, '/content/drive/MyDrive/공모전/open/SKT_kobert.model')

In [ ]:
# 테스트 문장 예측
dataset_test = pd.read_csv('/content/drive/MyDrive/공모전/open/test_otk_pororo_ver01.csv')
dataset_test['title'].fillna('', inplace=True)
dataset_test.isnull().sum()

index    0
title    0
dtype: int64

In [ ]:
class BERTDataset_Predict(Dataset):
    def __init__(self, dataset, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([title]) for title in dataset['title'].tolist()]
        # self.labels = [np.int32(label) for label in dataset['topic_idx'].tolist()]

    def __getitem__(self, i):
        return (self.sentences[i])

    def __len__(self):
        return (len(self.sentences))

In [ ]:
predict_data = BERTDataset_Predict(dataset_test, tok, max_len, True, False)

In [ ]:
predict_data.__getitem__(10)

(array([   2, 1209, 6664, 1132, 7207, 4427, 7207, 1263, 4427, 7207, 7003,
        6493, 4970, 4851, 1258, 7020, 1209, 6664, 4958, 7207, 4427, 7207,
        4427, 7207, 7003, 6493, 4970,    3,    1,    1], dtype=int32),
 array(28, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], dtype=int32))

In [ ]:
predict_data.__len__()

9131

In [ ]:
predict_dataloader = torch.utils.data.DataLoader(predict_data)

In [ ]:
predict_output = []
for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm_notebook(predict_dataloader)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  # print(out[0])
  predict_output.append(out[0].argmax())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
predict_output[10].cpu().numpy()

array(4)

In [ ]:
y_pred = []
for p in predict_output:
  y_pred.append(p.cpu().numpy())

In [ ]:
y_pred = np.array(y_pred)

In [ ]:
len(y_pred), type(y_pred[0])

(9131, numpy.int64)

In [ ]:
topic_dict = pd.read_csv("/content/drive/MyDrive/공모전/open/topic_dict.csv")

In [ ]:
submission = pd.read_csv("/content/drive/MyDrive/공모전/open/sample_submission.csv")
submission.topic_idx = y_pred
submission.head()

,index,topic_idx
0,45654,0
1,45655,3
2,45656,2
3,45657,0
4,45658,3


In [ ]:
%cd /content/drive/MyDrive/공모전/open

/content/drive/MyDrive/공모전/open


In [ ]:
submission.to_csv('baseline.csv',index = False)